## ハンズオン課題4 解答

### 4-1 解答例



#### 0. 準備

In [0]:
!pip install janome

#### 1. テキストの読み込みと分割

テキストファイル `joel_gtd.txt` を読み込み，改行コード `\n` で split して `sentences` 変数に格納する。

In [0]:
# janome-tutorial/hands-on/data/joel_gtd.txt を Google Colab にアップロード
from google.colab import files
uploaded = files.upload()

In [0]:
!cat -n joel_gtd.txt

In [0]:
with open('joel_gtd.txt') as f:
  data = f.read()
  
sentences = data.split('\n')
sentences

#### 2. トークナイズ

`sentences` の各行を形態素解析して，単語のリストのリストに変換して `texts` 変数に格納する。

In [0]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
texts = [t.tokenize(s, wakati=True) for s in sentences]
texts[0]

#### 3. Dictionary 作成

`texts` から，gensim の Dictionary を構築して `dictionary` 変数に格納する。

参考

-  gensim チュートリアル: https://radimrehurek.com/gensim/tut1.html


In [0]:
from gensim import corpora, models
dictionary = corpora.Dictionary(texts)
dictionary.token2id

#### 4. BoW 化と Corpus 作成

構築した `dictionary` を使って，`texts` を文書ベクトル (Bag of Words) に変換し，MmCorpus としてシリアライズする。

In [0]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('joel_gtd.mm', corpus)

!ls

#### 5. 文書ベクトル (BoW) から，出現回数の多い単語を調べる

MmCorpus を読み出し(デシリアライズ)，`corpus` 変数に格納する。テキストファイル 9 行目（「たとえば、あなたのチームの〜」）に出現する上位10単語とその出現回数を表示する。

- ヒント1: 9行目の文書ベクトルは， `corpus[8]` で取り出せる。
- ヒント2: 単語IDから単語（文字列）への変換は， `dictionary.get(id)` メソッドが使える。 

In [0]:
corpus = corpora.MmCorpus('joel_gtd.mm')
high_freq_words = sorted(corpus[8], key=lambda t: t[1], reverse=True)[:10]
[(dictionary.get(t[0]), t[1]) for t in high_freq_words]

#### 6. TFIDF モデル作成と適用

TfidfModel を作成して `tfidf` 変数に格納し，モデルを `corpus` に適用する。テキストファイル 9 行目に現れる単語の中で，tfidf 値の大きい上位10単語とその値を表示する。

ひとつ前の結果と比べてみましょう。

参考

- チュートリアル: https://radimrehurek.com/gensim/tut2.html

In [0]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
high_tfidf_words = sorted(corpus_tfidf[8], key=lambda t: t[1], reverse=True)[:10]
[(dictionary.get(t[0]), t[1]) for t in high_tfidf_words]

### 4-2 解答例

#### 0. 準備

In [0]:
!wget -q http://gensen.dl.itc.u-tokyo.ac.jp/soft/pytermextract-0_01.zip
!unzip -oq pytermextract-0_01.zip
!cd pytermextract-0_01 && python setup.py install --quiet
!pip freeze | grep termextract

#### 1. テキストファイルのアップロード

In [0]:
from google.colab import files
uploaded = files.upload()

#### 2. 複合名詞抽出＋出現回数カウント

In [0]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *


def wc(file):
    with open(file, encoding='utf8') as f:
        token_filters = [CompoundNounFilter(), POSKeepFilter('名詞,複合'), TokenCountFilter(sorted=True)]
        a = Analyzer(tokenizer=Tokenizer(), token_filters=token_filters)
        text = f.read()
        return list(a.analyze(text))
      

counts = wc('article.txt')
for k, v in counts:
  print('%s\t%d' % (k, v))